In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# 1. 数据预处理
# 读取数据
data = pd.read_excel(r'C:\Users\qyypy\Desktop\机器学习综合实践\Data\dataSet1\train.xlsx')


In [ ]:
# 删除缺失率过高的变量
def filter_nan(data, narate = 0.2):
    n_samples = data.shape[0]
    list_nan_cols = []
    
    for col in data.columns:
        if data[col].isna().sum() / n_samples >= (narate):
            list_nan_cols.append(col)
    
    print(f'缺失量在{narate * 100}%以上的变量有:{list_nan_cols}')

    return list_nan_cols

In [ ]:
list_nullfactor_todrop = filter_nan(data, narate=0.2)

df = data.drop(list_nullfactor_todrop, axis = 1).copy()
df = df.drop(columns='Patient Code')

df['Group '] = df['Group '].apply(lambda x: 0 if x == 'Control' else 1)
df.head()

In [ ]:
# KNN处理缺失值
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


numerical_cols = df.select_dtypes(include=['float64']).columns

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[numerical_cols])

knn_imputer = KNNImputer(n_neighbors=10)
imputed_data = knn_imputer.fit_transform(scaled_data)

imputed_data = scaler.inverse_transform(imputed_data)


df[numerical_cols] = imputed_data
df.info()
# df.to_csv('dataset1_preprocessed.csv', index=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# 选择特征和标签
X = df.drop(columns=['Group ']).copy()  # 确保没有多余的空格
y = df['Group ']

# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0, stratify=y)

# 转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# 2. 定义模型
class DNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x  

input_size = X_train.shape[1]
hidden_size = 64
output_size = 2  # 二分类任务

model = DNNModel(input_size, hidden_size, output_size)

# 3. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 训练模型
num_epochs = 2000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.numpy()

# 输出分类报告和混淆矩阵
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
torch.save(model, 'DNN.pt')

In [ ]:
DNN = torch.load(r'C:\Users\qyypy\Desktop\机器学习综合实践\Assignment\DNN.pt')

In [ ]:
data_test = pd.read_excel(r'C:\Users\qyypy\Desktop\机器学习综合实践\Test\Merge_Excel\train.xlsx')
data_test = data_test.drop(columns=['Total Protein', 'Albubin', 'Ferritin', 'PCT', 'IL-6', 'PT', 'Troponin', 'CPK-MB', 'LDH', 'Patient Code', 'Group '])
data_test

In [ ]:
# KNN处理缺失值
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

numerical_cols = data_test.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_test[numerical_cols])

knn_imputer = KNNImputer(n_neighbors=8)
imputed_data = knn_imputer.fit_transform(scaled_data)

imputed_data = scaler.inverse_transform(imputed_data)


data_test[numerical_cols] = imputed_data

data_test

In [ ]:
X1 = data_test[:][:456]
X1

In [ ]:
X_scaled = scaler.fit_transform(X1)
X_scaled

In [ ]:
Xpre = torch.tensor(X_scaled, dtype = torch.float32)
Xpre.shape

In [ ]:
DNN.eval()
with torch.no_grad():
    outputs = model(Xpre)
    _, predicted = torch.max(outputs, 1)
    ypre = predicted.numpy()

ypre

In [ ]:
ypre

In [ ]:
ypre.reshape(-1,1)

In [ ]:
df = pd.DataFrame({'id': [ i for i in range(1,457)], 'Group': ypre})
df

In [ ]:
df.to_csv('DNNprediction.csv',index = False)